In [ ]:
from platformtools.storage import PlatformStorage
from PIL import Image
from glob import glob
import seaborn as sns
from datetime import datetime
import pandas as pd
import numpy as np

In [ ]:
s3 = PlatformStorage()

In [ ]:
ds = s3.datasets.list()
ds

In [ ]:
s3.mount_bucket_for_object(ds[0])

In [ ]:
ds_fs_path = s3.get_mounted_path_for_object(ds[0])
ds_fs_path

In [ ]:
glob(f"{ds_fs_path}/*.jpg")

In [ ]:
Image.open(glob(f"{ds_fs_path}/*.jpg")[0])

In [ ]:
inferences = s3.inference.list()
i = inferences[0]
i

In [ ]:
s3.mount_bucket_for_object(i)
i_fs_path = s3.get_mounted_path_for_object(i)
i_fs_path
glob(f"{i_fs_path}/*.json")

In [ ]:
data = pd.read_json(glob(f"{i_fs_path}/*.json")[0], lines=False)
data

In [ ]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [ ]:
for col in data.columns:
    print(data[col][0])
    print(softmax(data[col][0]))

In [ ]:
benchmarks = s3.benchmarks.list()
b = benchmarks[0]
b

In [ ]:
s3.mount_bucket_for_object(b)
b_fs_path = s3.get_mounted_path_for_object(b)
b_fs_path
glob(f"{b_fs_path}/benchmark.json")

In [ ]:
benchmark_results = pd.read_json(glob(f"{b_fs_path}/benchmark.json")[0], lines=False)
benchmark_results

inference_results = pd.read_json(glob(f"{b_fs_path}/inference.json")[0], lines=False)
inference_results

In [ ]:
services = s3.services.list()
s = services[0]
s

In [ ]:
s3.mount_bucket_for_service(s)

In [ ]:
s3.services.get_definition(s)

In [ ]:
service_endpoints = s3.services.list_endpoints(s)
endpoint = service_endpoints[0]
service_endpoints

In [ ]:
service_calls = s3.services.list_calls(s, endpoint)
call = service_calls[0]
service_calls

In [ ]:
call_history = s3.services.list_call_history(s, endpoint, call)
date = call_history[0]
call_history

In [ ]:
s3.services.get_call_data(s, endpoint, call, date, "input")

In [ ]:
call_data = s3.services.get_call_data(s, endpoint, call, date)
weather = call_data['weather']
weather[0]

In [ ]:
weather_df = pd.DataFrame(weather)
weather_df['timestamp'] = pd.to_datetime(weather_df['timestamp'])
weather_df.head()

In [ ]:
p = sns.lineplot(x="timestamp", y="temperature", data=weather_df)
p.set_xticklabels(p.get_xticklabels(), rotation=30)